<a href="https://colab.research.google.com/github/Deciptikon/Family_parsing_from_site_RFPOISK/blob/main/Family_parsing_html_on_site_RFPOISK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/dm-fedorov/python_basic/blob/master/task_parsing_html.ipynb" target="_blank"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

Импорт бибилиотек

In [1]:
import time
import bs4  # импортируем Beautiful Soup
import re # регулярка для поиска цифр
import pandas as pd
from google.colab import files


### Функции

Читение сайта по URL

In [2]:
def read_url(url):
    import urllib.request
    with urllib.request.urlopen(url) as webpage:
        text = webpage.read().decode('utf-8')
    return text

Генерируем адрес страницы для чтения, по загатовке

In [19]:
def get_url_page(num_page: int = 0, min_num: int = 0, max_num: int = 100, family: str = 'bchkov/'):
  base_url = 'https://rfpoisk.ru/family/'
  prefix_page = '?page='
  if (min_num <= num_page) and (num_page <= max_num):
      return base_url + family + prefix_page + str(num_page)
  return ''

Функция парсинга, вытаскивает такую информацию из html-кода (класс "info") как:
*   Ссылку на профиль (для возможности уточнения\проверки);
*   Имя персоны;
*   День рождения (если он указан) в формате {ДД МЕСЯЦ ГГГГ};
*   Адрес в виде пары {СТРАНА, ГОРОД}, если указан.


In [58]:
# Постоянные элементы
base_url  = 'https://rfpoisk.ru/'

str_div_o = '<div class="info">'
str_a_o   = '<a '
str_href_o= 'href="'
# link
str_href_c= '">'
# name
str_a_c   = '</a>'
# if (exist)##########################
str_pclass= '<p class="birthday">'  ##
# birthday                          ##
str_p_c   = '</p>'####################
str_p_o   = '<p>'
# addres
str_p_c   = '</p>'
str_div_c = '</div>'


def data_from_class_info(info: str = ''):
  t2 = info[len(str_div_o + str_a_o + str_href_o) : len(t) - len(str_p_c + str_div_c)]

  s_url  = t2.find(str_href_c)
  s_name = t2.find(str_a_c)
  s_brt_o= t2.find(str_pclass)
  s_brt_c= t2.find(str_p_c)

  url    = base_url + t2[ : s_url]
  name   = t2[s_url + len(str_href_c) : s_name]
  brtd   = '_'
  adr    = ''

  if s_brt_o>0 :
    brtd = t2[s_name + len(str_a_c + str_pclass) : s_brt_c]
    adr  = t2[s_brt_c + len(str_p_o + str_p_c) : ]
  else :
    adr  = t2[s_name + len(str_a_c + str_p_o) : ]

  return url, name, brtd, adr

Загружаем страницы и вытаскиваем нужную информацию

In [59]:
max_num_page: int = 79          # максимальное количество страниц (например 79)
parse_family = 'bchkov/'        # Фамилия, берётся из адреса страницы
sleep_time = 10                 # задержка для уменьшения нагрузки на сайт

counter = 0
list_name = []
list_url = []
list_birthday = []
list_addres = []

for i in range(max_num_page):
  url_page = get_url_page(num_page=i, max_num=max_num_page, family=parse_family)
  print('=========================================================================================')
  print(f'== Page №{i}: {url_page}')
  print('=========================================================================================')
  print('')
  print('______________________________________')

  text = read_url(url_page) #текст загруженной страницы
  bs = bs4.BeautifulSoup(text, 'html.parser') #создаем парсер

  names = bs.find_all("div", {"class": "info"})

  for j in range(len(names)):
    t = str(names[j])
    u, n, b, a = data_from_class_info(t)

    list_url.append(u)
    list_name.append(n)
    list_birthday.append(b)
    list_addres.append(a)

    print(f'Persona №{counter}')
    print(f'URL:     {u}')
    print(f'Name:    {n}')
    print(f'Birtday: {b}')
    print(f'Addres:  {a}')
    print('______________________________________')
    print('')

    counter += 1

  time.sleep(sleep_time)

Выходные данные были обрезаны до нескольких последних строк (5000).
Birtday: 12 октября 0000 г.
Addres:  Беларусь, Минск
______________________________________

Persona №1751
URL:     https://rfpoisk.ru/people/tatyyana_bichkova233570981.html
Name:    Татьяна Бычкова
Birtday: 30 января 0000 г.
Addres:  Россия, Москва
______________________________________

Persona №1752
URL:     https://rfpoisk.ru/people/tatyyana_bichkova242474472.html
Name:    Татьяна Бычкова
Birtday: _
Addres:  Россия, Хабаровск
______________________________________

Persona №1753
URL:     https://rfpoisk.ru/people/tanya_bichkova243999451.html
Name:    Таня Бычкова
Birtday: 30 июня 0000 г.
Addres:  Россия, Москва
______________________________________

Persona №1754
URL:     https://rfpoisk.ru/people/tatyyana_bichkova254011966.html
Name:    Татьяна Бычкова
Birtday: 12 ноября 0000 г.
Addres:  Россия, Севастополь
______________________________________

Persona №1755
URL:     https://rfpoisk.ru/people/tatyyana_bichkova2

Сохраняем в таблицу эксель (в дальнейшем можно загрузить эту таблицу что бы не проводить полное сканирвоание повторно)

In [60]:
df = pd.DataFrame({
    'link' : list_url,
    'name': list_name,
    'birthday': list_birthday,
    'addres': list_addres
})
df.to_excel('info.xlsx') #'../info.xlsx'